## Make a request to a ChatGPT model

The following working example is from:  https://platform.openai.com/docs/quickstart?context=python


In [ ]:
# from openai import OpenAI
# client = OpenAI()
# 
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ]
# )

# print(completion.choices[0].message)

## Try generating related tweets

In [1]:
import pandas as pd
df_train = pd.read_csv("./data/train_clean_v03.csv")
# train_ids = df_train['id']
print(f"shape of base data with spillovers fixed and duplicates removed: {df_train.shape}")
df_train_class0 = df_train.loc[df_train['target'] == 0, ['id', 'text', 'target']]
df_train_class1 = df_train.loc[df_train['target'] == 1, ['id', 'text', 'target']]
print(f"shape of same base data with target == 0: {df_train_class0.shape}")
print(f"shape of same base data with target == 1: {df_train_class1.shape}")
print()

data = {'id': [], 'text': [], 'target': []}
df_aug_class0 = pd.DataFrame(data, columns = ['id', 'text', 'target'])
df_aug_class1 = pd.DataFrame(data, columns = ['id', 'text', 'target'])

for i, row in df_train_class1.iterrows():
    if i < 5:
        print(f"index={i} | {row['id']},{row['text']},{row['target']}")
    df_aug_class1.loc[len(df_aug_class1.index)] = [row['id'], row['text'], row['target']]

print(f"shape of df_aug_class1: {df_aug_class1.shape}")
print()
df_aug_class1.head(10)

shape of base data with spillovers fixed and duplicates removed: (7485, 5)
shape of same base data with target == 0: (4297, 3)
shape of same base data with target == 1: (3188, 3)

index=0 | 1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
index=1 | 4,Forest fire near La Ronge Sask. Canada,1
index=2 | 5,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
index=3 | 6,13,000 people receive #wildfires evacuation orders in California ,1
index=4 | 7,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ,1
shape of df_aug_class1: (3188, 3)



,id,text,target
0,1,Our Deeds are the Reason of this #earthquake M...,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are ...,1
3,6,"13,000 people receive #wildfires evacuation or...",1
4,7,Just got sent this photo from Ruby #Alaska as ...,1
5,8,#RockyFire Update => California Hwy. 20 closed...,1
6,10,#flood #disaster Heavy rain causes flash flood...,1
7,13,I'm on top of the hill and I can see a fire in...,1
8,14,There's an emergency evacuation happening now ...,1
9,15,I'm afraid that the tornado is coming to our a...,1


In [2]:
# df_train_class0.head(22)
base_tweet_class0 = df_train_class0['text'].loc[df_train_class0['id'] == 57].values[0]
base_tweet_class1 = df_train_class1['text'].loc[df_train_class1['id'] == 56].values[0]
print(base_tweet_class0)
print(base_tweet_class1)

Ablaze for you Lord :D
Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J


In [3]:
import projtools as pt
import pandas as pd

df_prompt_data = pd.read_csv("./data/prompt_log.csv")
df_prompt_data
# get current context and prompt prefix
# context, start_prompt_class0 = pt.get_prompt_setup(prompt_date='2024-02-28', target_class=0)

,date,prompt_component,class,content,notes
0,2024-02-26,prompt_prefix,0,"Write me a tweet similar to this one, under 14...",initial class 0 prompt used in web interface
1,2024-02-26,prompt_prefix,1,"Write me a tweet similar to this one, under 14...",initial class 1 prompt used in web interface
2,2024-02-27,context,2,You are a fiction writer who has observed a di...,initial context used sending API requests from...
3,2024-02-27,prompt_prefix,0,"Write me a tweet similar to this one, under 14...",avoid having to expand contractions on generat...
4,2024-02-27,prompt_prefix,1,"Write me a tweet similar to this one, under 14...",avoid having to expand contractions on generat...
5,2024-02-29,prompt_prefix,0,"Write me a tweet similar to this one, under 14...",fix issue with generated tweet quoting
6,2024-02-29,prompt_prefix,1,"Write me a tweet similar to this one, under 14...",fix issue with generated tweet quoting


In [4]:
# set up a single request
# context = "You are a fiction writer who has observed a disaster and likes to tweet."
# start_prompt_class0 = "Write me a tweet similar to this one, under 141 characters, " + \
#                       "does not use contractions, but refers to a different activity and location: "  # "...activity...
# start_prompt_class1 = "Write me a tweet similar to this one, under 141 characters, " + \
#                       "does not use contractions, but refers to a different disaster and location: "  # "...disaster...
current_prompt = pt.get_prompt_setup()
context = current_prompt['context']['text']
start_prompt_class0 = current_prompt['prefix_class0']['text']
start_prompt_class1 = current_prompt['prefix_class1']['text']

complete_prompt_class0 = start_prompt_class0 + base_tweet_class0
complete_prompt_class1 = start_prompt_class1 + base_tweet_class1
print(complete_prompt_class0)
print(complete_prompt_class1)

Write me a tweet similar to this one, under 141 characters, does not contain contractions or double quotes, but refers to a different activity and location: Ablaze for you Lord :D
Write me a tweet similar to this one, under 141 characters, does not contain contractions or double quotes, but refers to a different disaster and location: Barbados #Bridgetown JAMAICA ÛÒ Two cars set ablaze: SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintende...  http://t.co/wDUEaj8Q4J


In [ ]:
# test batch of 3 tweets
# df_aug_class1_test3 = df_aug_class1.iloc[:3]
# print(context)
# df_aug_class1_test3

In [ ]:
# df_train['id'].max()  # 10873, add 20,000 to id of augmented samples

In [5]:
# batch_range = (0, 100)
# batch_range = (100, 200)
batch_range = (200, 300)

In [6]:
from openai import OpenAI

client = OpenAI()

aug_offest = 20000
aug_tweets_class0 = {}
aug_tweets_class1 = {}
df_train_chunk_class0 = df_train_class0.iloc[batch_range[0]:batch_range[1]]
df_train_chunk_class1 = df_train_class1.iloc[batch_range[0]:batch_range[1]]

In [7]:
df_train_chunk_class0.head()

,id,text,target
335,486,Tomorrow is the day we start armageddon #prese...,0
336,487,Lee does comedy: ÛÏ@LeeJasper: Working class ...,0
337,489,9 Charts Prove Financial Crisis Part 2 Has BEG...,0
339,492,@RohnertParkDPS You're on stage now! Right und...,0
340,493,**OFFICIAL VID** #TheReal &gt;&gt;&gt; https:/...,0


In [8]:
df_train_chunk_class1.head()

,id,text,target
488,713,Christian Attacked by Muslims at the Temple Mo...,1
489,714,#PT: The unit attacked by IS was responsible f...,1
490,716,Telnet attacked from 124.13.172.40 (STREAMYX-H...,1
491,717,Christian Attacked by Muslims at the Temple Mo...,1
495,724,Christian Attacked by Muslims at the Temple Mo...,1


In [9]:
import time

# generate (next) 100 class 0 tweets
t0 = time.time()
rows_processed = 1
for i, row in df_train_chunk_class0.iterrows():
    prompt_content = start_prompt_class0 + row['text']
    gen_tweet = pt.get_aug_tweet(context, prompt_content)
    aug_id = row['id'] + aug_offest
    aug_tweets_class0[aug_id] = gen_tweet
    if rows_processed % 10 == 0:
        print(f"processing row {rows_processed} with id {row['id']}")
    rows_processed += 1
t1 = time.time()

processing row 10 with id 501
processing row 20 with id 521
processing row 30 with id 535
processing row 40 with id 564
processing row 50 with id 609
processing row 60 with id 624
processing row 70 with id 642
processing row 80 with id 687
processing row 90 with id 720
processing row 100 with id 752


In [10]:
gen_tweet_count = 100
print(f"time to do {gen_tweet_count} is {(t1-t0)/60.} minutes")

time to do 100 is 2.826367215315501 minutes


In [14]:
out_file_name0 = f"./data/aug_tweets_class0_v01prompt_{batch_range[0]:04}_{(batch_range[1]-1):04}.csv"
print(out_file_name0)

./data/aug_tweets_class0_v01prompt_0200_0299.csv


In [15]:
pt.write_aug_tweets(aug_tweets_class0, 0, out_file_name0)

True

In [16]:
# generate (next) 100 class 1 tweets
t0 = time.time()
rows_processed = 1
for i, row in df_train_chunk_class1.iterrows():
    prompt_content = start_prompt_class1 + row['text']
    gen_tweet = pt.get_aug_tweet(context, prompt_content)
    aug_id = row['id'] + aug_offest
    aug_tweets_class1[aug_id] = gen_tweet
    if rows_processed % 10 == 0:
        print(f"processing row {rows_processed} with id {row['id']}")
    rows_processed += 1
t1 = time.time()
print(f"time to do {gen_tweet_count} is {(t1-t0)/60.} minutes")
out_file_name1 = f"./data/aug_tweets_class1_v01prompt_{batch_range[0]:04}_{(batch_range[1]-1):04}.csv"
pt.write_aug_tweets(aug_tweets_class1, 1, out_file_name1)

processing row 10 with id 731
processing row 20 with id 791
processing row 30 with id 837
processing row 40 with id 863
processing row 50 with id 883
processing row 60 with id 902
processing row 70 with id 945
processing row 80 with id 1122
processing row 90 with id 1269
processing row 100 with id 1409
time to do 100 is 4.507935984929403 minutes


True

In [ ]:
import pickle

pickle_file = './data/first3_class1.pickle'
# Store data (serialize) - commented out because it's already be done so just need to read
with open(pickle_file, 'wb') as handle:
    pickle.dump(aug_responses, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Load data (deserialize)
with open(pickle_file, 'rb') as handle:
    aug_responses = pickle.load(handle)

# print(aug_responses == unserialized_data)
print(aug_responses)

In [ ]:


print(aug_responses.keys())
for key in aug_responses.keys():
    cgpt_response = aug_responses[key].choices[0].message.content
    print(cgpt_response)